In [ ]:
# Add project root to PYTHONPATH
import sys
import os

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root directory to the PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify if the project root directory is in PYTHONPATH
print("Project root added to PYTHONPATH:", project_root in sys.path)

# Import necessary libraries and classes
from open_science_agent.api.semantic_scholar_client import SemanticScholarClient
from open_science_agent.api.notion_client import NotionClient
# from open_science_agent.data_processing.paper_analyzer import PaperAnalyzer
# from open_science_agent.data_processing.paper_organizer import PaperOrganizer

# Initialize the SemanticScholarClient
semantic_scholar_client = SemanticScholarClient()

# Define the search query
query = "machine learning in atmospheric science"

# Perform the search by relevance
results = semantic_scholar_client.search_papers_by_relevance(query)
